# Harry Potter Sentiment-Analyse

In dieser Arbeit werden die Originaltexte von Harry Potter sowie verschiedener
Fan-Fiction-Texte behandelt.

Mittels einer Netzwerkanalyse werden die Beziehungen zwischen den handelnden 
Personen untersucht.
Dabei wird die Python-Bibliothek SpaCy verwendet, um die *Named Entity
Recognition* durchzuführen.

Anschließend werden die gefundenen Namen mit Variationen davon gematcht.
Beispielsweise werden "Harry" und "Harry Potter" nicht als eine Entity erkannt.

Darauffolgend sollen die Interaktionen zwischen handelnden Personen erkannt 
werden.
Dabei wird untersucht, ob ein Satz mehrere Figuren erwähnt.
Mittels des BERT-Modells wird eine Sentimentanalyse durchgeführt, um das 
Sentiment der einzelnen Sätze zu untersuchen.

Gibt es in dem Satz zwei handelnde Personen, dann wird der zugehörige 
Sentiment-Score auf die jeweilige Beziehung zwischen zwei Figuren addiert.

Der ermittelte Graph wird visuell dargestellt.
Dabei entsprechen die Knoten einzelnen Figuren, die Kanten den Beziehungen der 
Figuren untereinander.

Knoten werden je nachdem, ob sie positiv oder negativ beschrieben werden,
eher grün (positiv) oder rot (negativ) dargestellt.
Die Kanten werden dicker, je mehr Interaktionen es zwischen den Figuren gibt.
Auch diese werden je nachdem, ob es sich um positive oder negative Interaktionen
handelt, eher grün (positiv) oder eher rot (negativ) eingefärbt.


In [1]:

'''
Eine Figur.
- Name, unter der die Person referenziert wird
- Aliase, unter der eine Person auch bekannt ist
- einer Liste von Sätzen, in denen die Person alleine vorkommt
- ein Sentiment Value, der aus dem Sentiment der einzelnen Sätze ermittelt wird.
'''
class Figure:

	name = ""
	sentences = []
	sentiment_value = 0

	def __init__(self, name: str) -> None:
		self.name = name
		self.aliases = [name]

In [2]:
from typing import List

'''
Eine Beziehung zwischen zwei oder mehr Figuren.
- Satz, in dem zwei oder mehr Personen erwähnt werden
- Sentiment Score, der aus dem Satz berechnet wird
- Figuren, die in dem Satz erwähnt werden
'''
class Relation:

	sentence = None
	sentiment_score = 0
	figures = List[Figure]

	def __init__(self, sentence, sentiment_score, figures: List[Figure]) -> None:
		self.sentence = sentence
		self.sentiment_score = sentiment_score
		self.figures = figures

In [3]:
from typing import List

'''
Eine Story, die handelnde Personen und Beziehungen zusammenfasst.
- Name der Story
- Figuren der Story
- Beziehungen zwischen den Figuren
- Ein SpaCy-Objekt, das den gesamten Text vorhält
- Der Text ohne Front- und Backmatter
'''
class Story:
	name = ""
	figures = {}
	relations = []
	nlp = None
	stripped_text = ""

	def __init__(self, name, nlp, stripped_text) -> None:
		self.name = name
		self.nlp = nlp
		self.stripped_text = stripped_text

	def add_figure(self, figure: Figure) -> None:
		self.figures[figure.name] = figure

	def add_relation(self, relation: Relation) -> None:
		self.relations.append(relation)

	def add_sentence_to_figure(self, name, sentence, score):
		self.figures[name].sentences.append((sentence, score))

## Laden der Bücher oder Fan-Fictions

Im Folgenden können die Texte geladen werden, auf denen die Analyse ausgeführt wird.
Dazu müssen sich die Dateien im Unterordner `data` befinden und wie folgt heißen:

|Buch|Dateiname|
|----|---------|
|Harry Potter und der Stein der Weisen|Harry Potter und der Stein der Weisen (German Edition).txt|
|Harry Potter und die Kammer des Schreckens|Harry Potter und die Kammer des Schreckens (German Edition).txt|
|Harry Potter und der Gefangene von Askaban|Harry Potter und der Gefangene von Askaban (German Edition).txt|
|Harry Potter und der Feuerkelch|Harry Potter und der Feuerkelch (German Edition).txt|
|Harry Potter und der Orden des Phönix|Harry Potter und der Orden des Phnix  5 (German Edition).txt|
|Harry Potter und der Halbblutprinz|Harry Potter und der Halbblutprinz (German Edition).txt|
|Harry Potter und die Heiligtümer des Todes|Harry Potter und die Heiligtmer des Todes (German Edition).txt|
|Harry Potter und das verwunschene Kind|Harry Potter und das verwunschene Kind. Teil eins und zwei (Bhnenfassung) (German Edition).txt|
|Quidditch im Wandel der Zeiten|Quidditch im Wandel der Zeiten (Hogwarts Schulbcher) (German Edition).txt|
|Die Marchen von Beedle dem Barden|Die Marchen von Beedle dem Barden (Hogwarts Schulbcher) (German Edition).txt|
|Hogwarts - Ein unvollständiger und unzuverlässiger Leitfaden|Hogwarts  Ein unvollstndiger und unzuverlssiger Leitfaden (Kindle Single) (Pottermore Presents) (German Edition).txt|
|Kurzgeschichten aus Hogwarts  Heldentum, Härtefälle und hanebüchene Hobbys|Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys (Kindle Single) (Pottermore Presents) (German Edition).txt|
|Kurzgeschichten aus Hogwarts  Macht, Politik und nervtötende Poltergeister|Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister (Kindle Single) (Pottermore Presents) (German Edition).txt|
|Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch|Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch (German Edition).txt|
|Phantastische Tierwesen und wo sie zu finden sind|Phantastische Tierwesen und wo sie zu finden sind (Hogwarts Schulbcher) (German Edition).txt|

|Fan-Fiction|Dateiname|
|----|---------|
|FF1|ff1.txt|

In [4]:
import spacy

sp = spacy.load("de_core_news_lg")

'''
Eine Story laden
Übergeben werden der Name sowie der Dateinamen einer Story
Außerdem können zu Beginn Zeilen übersprungen werden (skip_lines), um bspw. das Frontmatter zu überspringen und
am Ende lassen sich Zeilen abschneiden (trim_lines), um bspw. das Backmatter abzuschneiden
'''
def load_story(name, filename, skip_lines=0, trim_lines=0) -> Story:
	nlp = None
	with open("./data/" + filename, "r") as f:
		text = f.read()
		text = "\n".join(text.split("\n")[skip_lines:trim_lines])
		text = text.replace("»", "\"").replace("«", "\"").replace("›", "\"").replace("‹", "\"").replace("…", "")
		text = "\n".join([l.strip("- \n") for l in text.split("\n")]).replace("\n\n", "\n")
		nlp = sp(text)
	return Story(name, nlp, text)

/home/panda/projects/harry-potter-sentiment/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Laden der Bücher

story = load_story("Harry Potter und der Stein der Weisen", "Harry Potter und der Stein der Weisen (German Edition).txt", skip_lines=53, trim_lines=6260)
# story = load_story("Harry Potter und die Kammer des Schreckens", "Harry Potter und die Kammer des Schreckens (German Edition).txt")
# story = load_story("Harry Potter und der Gefangene von Askaban", "Harry Potter und der Gefangene von Askaban (German Edition).txt")
# story = load_story("Harry Potter und der Feuerkelch", "Harry Potter und der Feuerkelch (German Edition).txt")
# story = load_story("Harry Potter und der Orden des Phönix", "Harry Potter und der Orden des Phnix  5 (German Edition).txt")
# story = load_story("Harry Potter und der Halbblutprinz", "Harry Potter und der Halbblutprinz (German Edition).txt")
# story = load_story("Harry Potter und die Heiligtümer des Todes", "Harry Potter und die Heiligtmer des Todes (German Edition).txt")
# story = load_story("Harry Potter und das verwunschene Kind", "Harry Potter und das verwunschene Kind. Teil eins und zwei (Bhnenfassung) (German Edition).txt")
# story = load_story("Quidditch im Wandel der Zeiten", "Quidditch im Wandel der Zeiten (Hogwarts Schulbcher) (German Edition).txt")
# story = load_story("Die Marchen von Beedle dem Barden", "Die Marchen von Beedle dem Barden (Hogwarts Schulbcher) (German Edition).txt")
# story = load_story("Hogwarts - Ein unvollständiger und unzuverlässiger Leitfaden", "Hogwarts  Ein unvollstndiger und unzuverlssiger Leitfaden (Kindle Single) (Pottermore Presents) (German Edition).txt")
# story = load_story("Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys", "Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys (Kindle Single) (Pottermore Presents) (German Edition).txt")
# story = load_story("Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister", "Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister (Kindle Single) (Pottermore Presents) (German Edition).txt")
# story = load_story("Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch", "Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch (German Edition).txt")
# story = load_story("Phantastische Tierwesen und wo sie zu finden sind", "Phantastische Tierwesen und wo sie zu finden sind (Hogwarts Schulbcher) (German Edition).txt")

# TODO: Fan-Fictions raussuchen

## Charakter-Mapping erzeugen

Basierend auf der [Liste von Charakteren in Übersetzungen von Harry Potter](https://harrypotter.fandom.com/de/wiki/Liste_von_Charakteren_in_%C3%9Cbersetzungen_von_Harry_Potter), die in die Datei `data/character-mapping.txt` übersetzt wurde, wird eine [SpaCy-KnowledgeBase](https://spacy.io/api/kb/) aufgebaut.
Diese KnowledgeBase besteht aus [Candidates](https://spacy.io/api/kb/#candidate), die wiederum einen oder mehrere Aliase verwenden.

Für jede Entität wird ein Entitätsvektor erzeugt.
Da die Entität eindeutig identifiziert werden kann, wird dieser Entitätsvektor mit einer Eins für die Entität und  Nullen für alle anderen Identitäten verwendet.

In [6]:
# Charakter-KnowledgeBase erstellen

from spacy.kb import KnowledgeBase
import re

entity_matcher = re.compile("^([^\t].+?)( \((.+?)\))?$")
last_character = ""

characters = {}

# Character-Mapping einlesen
with open("./data/character-mapping.txt", "r") as f:
	for line in f:
		is_entity = not line.startswith("\t")
		if is_entity:
			m = entity_matcher.match(line.strip())
			name = m.group(1)
			tags = [t.strip() for t in m.group(3).split(",")] if m.group(3) is not None else []
			last_character = name
			characters[name] = {"tags": tags, "aliases": []}
		else:
			alias = line.strip()
			characters[last_character]["aliases"].append(alias)

num_characters = len(characters)
vector_counter = 0
for e in characters:
	char_vector = ([0] * vector_counter) + [1] + ([0] * (num_characters - 1 - vector_counter))
	vector_counter += 1
	characters[e]["vector"] = char_vector

# Knowledge-Base erzeugen
doc = story.nlp
vocab = doc.vocab
stripped_text = story.stripped_text

def count_occurences(name):
	return stripped_text.count(name)

kb = KnowledgeBase(vocab=vocab, entity_vector_length=num_characters)
for char in characters:
	char_firstname = char.split(" ")[0]
	char_freq = count_occurences(char_firstname)

	ent = kb.add_entity(entity=char, freq=char_freq, entity_vector=characters[char]["vector"])
	
	kb.add_alias(alias=char, entities=[char], probabilities=[1])
	if char != char_firstname:
		kb.add_alias(alias=char_firstname, entities=[char], probabilities=[1])
	
	for alias in characters[char]["aliases"]:
		kb.add_alias(alias=alias, entities=[char], probabilities=[1])

In [7]:
# Teste Mapping für verschiedene Charaktere:

search_characters = ["Harry", "Harry Potter", "Ron", "Ron Weasley", "Hermine", "Hermine Granger", "Draco", "Draco Malfoy", "Voldemort", "Du-weißt-schon-wer"]

for sc in search_characters:
	print("Suche nach \"{}\"".format(sc))
	candidates = kb.get_alias_candidates(sc)
	for c in candidates:
		print("\tEntity:", c.entity_)
		print("\tAlias :", c.alias_)


Suche nach "Harry"
	Entity: Harry Potter
	Alias : Harry
Suche nach "Harry Potter"
	Entity: Harry Potter
	Alias : Harry Potter
Suche nach "Ron"
	Entity: Ron Weasley
	Alias : Ron
Suche nach "Ron Weasley"
	Entity: Ron Weasley
	Alias : Ron Weasley
Suche nach "Hermine"
	Entity: Hermine Granger
	Alias : Hermine
Suche nach "Hermine Granger"
	Entity: Hermine Granger
	Alias : Hermine Granger
Suche nach "Draco"
	Entity: Draco Malfoy
	Alias : Draco
Suche nach "Draco Malfoy"
	Entity: Draco Malfoy
	Alias : Draco Malfoy
Suche nach "Voldemort"
	Entity: Lord Voldemort
	Alias : Voldemort
Suche nach "Du-weißt-schon-wer"
	Entity: Lord Voldemort
	Alias : Du-weißt-schon-wer


In [8]:
# Return an entity for a name
def find_entity(name):
	ent_str = str(name).strip(' .,:!?"')
	candidates = kb.get_alias_candidates(ent_str)
	if len(candidates) > 0:
		return candidates[0]
	elif ent_str.endswith("s"):
		# Genitiv-S entfernen
		return find_entity(ent_str[:-1])
	elif " " in ent_str:
		candidates = [c for c in (find_entity(w) for w in ent_str.split(" ")) if c != None]
		if len(candidates) > 0:
			return candidates[0]
	return None


## Figuren erzeugen

Die Figuren aus der Knowledge-Base werden als Figuren in die Story überführt.

In [9]:
for char in kb.get_entity_strings():
	story.add_figure(Figure(char))

print([f for f in story.figures])

['Winky', 'Rubeus Hagrid', 'Violet', 'Kingsley Shacklebolt', 'Lily Evans', 'Kreacher', 'Rolanda Hooch', 'Nymphadora Tonks', 'Ron Weasley', 'Terry Boot', 'Magdalene "Magda" Dursley', 'Sirius Black', 'Silvanus Kesselbrand', 'Norberta', 'Cadogan', 'Marcus Flint', 'Malcolm Baddock', 'Firenze', 'Viktor Krum', 'Hedwig', 'Irma Pince', 'Cuthbert Binns', 'Wilhelmina Raue-Pritsche', 'Barty Crouch sr.', 'Charity Burbage', 'Mandy Brocklehurst', 'Bill Weasley', 'Petunia Dursley', 'Harry Potter', 'Griphook', 'Emmeline Vance', 'Septima Vektor', 'Argus Filch', 'Charlie Weasley', 'Amycus Carrow', 'Colin Creevey', 'Peeves', 'Bertie Bott', 'Gabrielle Delacour', 'George Weasley', 'Mad-Eye Moody', 'Mr. Ollivander', 'Quirinus Quirrell', 'Sybill Trelawney', 'Arnold Friedlich', 'Graue Dame', 'Der Blutige Baron', 'Dudley Dursley', 'Krätze', 'R.A.B.', 'Wilbert Gimpel', 'Narzissa Malfoy', 'Luna Lovegood', 'Neville Longbottom', 'Albus Dumbledore', 'Olympe Maxime', 'Fang', 'Severus Snape', 'Mrs Norris', 'Bane', 'A

## Sentiment Analysis

In den nächsten Code-Blöcken werden zwei Aufgaben erledigt.

Zum einen werden die einzelnen Charaktere analysiert, zum anderen werden auch ihre Beziehungen zueinander modelliert.

### Sentiment-Analyse einzelner Charaktere



In [10]:
from transformers import pipeline, logging
logging.set_verbosity_error()

classifier = pipeline(task='sentiment-analysis', model="bert-base-german-cased") #bert-base-german-dbmdz-cased 

In [13]:
import sys
from tqdm.auto import tqdm

num_sents = sum(1 for _ in (enumerate(doc.sents)))

for f in story.figures:
	story.figures[f].sentences = []

pbar = tqdm(total=num_sents)
for s in doc.sents:
	entities = set()
	for e in s.ents:
		if (e.label_ == "PER"):
			entity = find_entity(e.text)
			if entity != None:
				entities.add(entity)

	num_entities = len(entities)
	if num_entities == 0:
		# Überspringen, da keine Entität erkannt wird
		a = True
	elif num_entities == 1:
		# figure out how to calculate sentiment for a person
		name = next(iter(entities)).entity_
		score = classifier(s.text)[0]["score"]
		story.add_sentence_to_figure(name, s.text, score)
		file = open("./out/{}.csv".format(name), "a")
		file.write(str(s).strip() + "\t" + str(score) + "\n")
		file.close()
	elif num_entities >= 2:
		# figure out how to calculate sentiment for multiple people
		a = True
	pbar.update(1)

100%|██████████| 5756/5756 [03:33<00:00, 26.90it/s]


In [12]:
# Berechne den Wert für den persönlichen Sentiment Score
for f in story.figures:
	num_sents = len(story.figures[f].sentences)
	if num_sents == 0:
		continue
	sum_sents = sum(s[1] for s in story.figures[f].sentences)
	story.figures[f].sentiment_value = sum_sents / num_sents
	print("{},{},{},{}".format(f, num_sents, sum_sents, story.figures[f].sentiment_value))

Rubeus Hagrid,149,92.52840393781662,0.6209959995826618
Rolanda Hooch,9,5.575520992279053,0.6195023324754503
Ron Weasley,154,93.19435912370682,0.6051581761279663
Sirius Black,1,0.5788446068763733,0.5788446068763733
Marcus Flint,3,1.8016930222511292,0.6005643407503763
Firenze,1,0.5769597887992859,0.5769597887992859
Hedwig,10,6.114540755748749,0.6114540755748749
Irma Pince,1,0.598721444606781,0.598721444606781
Petunia Dursley,36,22.577421963214874,0.6271506100893021
Harry Potter,265,161.07495206594467,0.6078300077960176
Griphook,5,3.0752639174461365,0.6150527834892273
Argus Filch,26,15.764817893505096,0.606339149750196
Charlie Weasley,4,2.4191972613334656,0.6047993153333664
Peeves,13,7.884963929653168,0.6065356868963975
Bertie Bott,3,1.7399808764457703,0.5799936254819235
George Weasley,7,4.2893083691596985,0.6127583384513855
Dudley Dursley,64,38.34395486116409,0.599124294705689
Krätze,3,1.9125356078147888,0.6375118692715963
Neville Longbottom,39,23.717630982398987,0.6081443841640766
Albus

100%|██████████| 5756/5756 [02:30<00:00, 35.96it/s]

### Sentiment-Analyse von Beziehungen zwischen zwei Charakteren